In [18]:
#basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

import time

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split


In [19]:
df = pd.read_csv('data/DJIA_table_with_Label.csv')

In [21]:
print(df.shape[0])
df

1989


,Unnamed: 0,Date,Open,High,Low,Close,Volume,Adj Close,date_obj,Label,Label5,Close5,Label20,Close20,Label60,Close60
0,0,2008-08-08,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,2008-08-08,NaN,1.0,11659.900391,1.0,11510.740234,0.0,9319.830078
1,1,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,2008-08-11,1.0,0.0,11479.389648,0.0,11230.730469,1.0,9625.280273
2,2,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,2008-08-12,0.0,0.0,11348.549805,1.0,11268.919922,0.0,9139.269531
3,3,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,2008-08-13,0.0,1.0,11417.429688,1.0,11433.709961,0.0,8695.790039
4,4,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,2008-08-14,1.0,1.0,11430.209961,0.0,11421.990234,1.0,8943.809570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,1984,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,2016-06-27,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1985,1985,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,2016-06-28,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1986,1986,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,2016-06-29,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1987,1987,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,2016-06-30,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def create_dataset(df,attribsSet,attribIndex, prediction_cols,pindex=0,window=10 ):
    x = []
    y = []
    xInput = df[attribsSet[attribIndex]].values
    yInput = df[prediction_cols[pindex]].values
    #print("x",x_in.shape,"y : ", y_in.shape[0])
    for index in df.index:
        if index >= window:
           x.append(xInput[index-window:index].flatten())
           y.append(yInput[index])
    x = np.array(x)
    y = np.array(y)            
    #print(x.shape, y.shape)
    return x,y

In [23]:
def create_dataset_reg(df,attribsSet,attribIndex, prediction_cols,pindex=0,label_col='Label',window=10 ):
    x = []
    y = []
    z = []
    xInput = df[attribsSet[attribIndex]].values
    yInput = df[prediction_cols[pindex]].values
    zInput = df[label_col].values
    #print("x",x_in.shape,"y : ", y_in.shape[0])
    for index in df.index:
        if index >= window:
           x.append(xInput[index-window:index].flatten())
           y.append(yInput[index])
           z.append(zInput[index])
    x = np.array(x)
    y = np.array(y)            
    z = np.array(z)
    #print(x.shape, y.shape)
    return x,y,z

In [24]:
def save_results_to_csv(results, colnames, file_name_suffix ):
    results_df = pd.DataFrame(results,columns=colnames)
    fname=file_name_suffix+str(time.time())+'.csv'
    results_df.to_csv(fname)

In [25]:
def direction(price_tplusone, price_t):
    if price_tplusone > price_t :
        return 1
    return 0

In [36]:
def accuracy_of_direction(y_predicted,x_test,y_test,attribCols,indexOfPrice=0):
#def accuracy_of_direction(y_predicted, label_test,price_test):                
                collen = len(attribCols)                
                label_predicted = []
                label_test = []
                match_count=0                
                for index in range(0,len(y_predicted)):                     
                     l = len(x_test[index])
                     prev_price = x_test[index][l-collen-1:][indexOfPrice]
                     pred_l=direction(y_predicted[index], prev_price)
                     deriv_l=direction(y_test[index], prev_price)   
                     label_predicted.append(pred_l)  
                     label_test.append(deriv_l)
                     match='N'   
                     if pred_l == deriv_l:
                            match_count=match_count+1
                            match='Y'
                     #print(y_predicted[index],prev_price,y_test[:,0][index],match)   
                    
                ts_acc_pca = accuracy_score(label_test,label_predicted,normalize='false')
                print('Score',ts_acc_pca,match_count,len(label_test))
                return ts_acc_pca

In [33]:
def run_predictions_classifier(df, prediction_cols = [['Label']],
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],
                              train_percentages=[0.2,0.3,0.5,0.6],
                              window_sizes = [5,10,15,20,30,40,50,55,60,65,70,80,100], 
                              csvPrefix='results_classification'
                              ): 
    
    resultsList=['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']    
    modelList = []
    modelList.append(("LogisticReg",LogisticRegression(random_state=42)))
    modelList.append(("RandomForestClassifier",RandomForestClassifier(random_state=42)))
    modelList.append(("SVC",SVC(random_state=42)))
    modelList.append(("DecisionTreeClassifier",DecisionTreeClassifier(random_state=42)))
    modelList.append(("KNeighborsClassifier",KNeighborsClassifier(10)))
    modelList.append(("GaussianNB",GaussianNB()))
    modelList.append(("XGBClassifier",XGBClassifier()))
    modelList.append(("AdaBoostClassifier",AdaBoostClassifier()))    
    ts_acc_best=-2.0
    best=[]    
    
    for attribIndex in range(0,len(attribsSet)):    
        closeVoldf = df[attribsSet[attribIndex]].values                                                
        for train_per in train_percentages:            
            for window in window_sizes:
                
                X,Y = create_dataset(df,attribsSet,attribIndex, prediction_cols,pindex=0,window=window)  
                Y = Y.ravel()
                print(X.shape,Y.shape)
                
                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                X = min_max_scaler.fit_transform(X)                
                                
                x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=train_per, random_state=42)
                for name, classifier in modelList:                                      
                    classifier.fit(x_train,y_train)                    
                    yts_pred = classifier.predict(x_test)
                    ts_acc= accuracy_score(y_test,yts_pred)                        
                    print('Completed ',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc)
                    resultsList.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                    if(ts_acc_best <= ts_acc) :
                        ts_acc_best = ts_acc
                        best = [train_per,window,name,attribsSet[attribIndex],'No PCA']

                    pca_final = PCA(0.95)
                    df_train_pca = pca_final.fit_transform(x_train)
                    df_test_pca = pca_final.transform(x_test)

                    classifier.fit(df_train_pca,y_train)                    
                    yts_pred_pca = classifier.predict(df_test_pca)
                    
                    ts_acc_pca = accuracy_score(y_test,yts_pred_pca)

                    resultsList.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])
                    print('Completed with PCA with ',pca_final.n_components_,' components',train_per,window,name,attribsSet[attribIndex]," Score[test]",ts_acc_pca)
                    if(ts_acc_best <= ts_acc_pca) :
                        ts_acc_best = ts_acc_pca
                        best = [train_per,window,name,attribsSet[attribIndex],'With PCA']
    print('Best',ts_acc_best,best)          
    save_results_to_csv(resultsList[6:],['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy'],csvPrefix)

In [ ]:

# (df, prediction_cols = [['Label']],
#                               attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],
#                               train_percentages=[0.2,0.3,0.5,0.6],
#                               window_sizes = [5,10,15,20,30,40,50,55,60,65,70,80,100], 
#                               csvPrefix='results_classification'
#                               )
run_predictions_classifier(df,prediction_cols = [['Label']],attribsSet =[['Close','Volume','High','Low']],
                           train_percentages=[0.6],
                           window_sizes = [60],
                          csvPrefix='results_classification_tplus1_')



In [ ]:
x,y,z = create_dataset_reg(df,attribsSet=[['Date','Close','Volume']],attribIndex=0,
                            prediction_cols=[['Close','Label']],pindex=0,label_col='Label',window=3 )
print(x[0:3])
#min_max_scaler = MinMaxScaler(feature_range=(0,1))
#y = min_max_scaler.fit_transform(y) 
#x = min_max_scaler.fit_transform(x) 
#print(x[0:3])
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.6, random_state=42)
print(x_test[5])
attribsSet=[['Close','Volume']]
attribIndex=0
attribCols = attribsSet[attribIndex]
collen = len(attribCols)
colOfPrice=1
l = len(x_test[5])
print(x_test[5][l-collen-1:][colOfPrice])

print(y_test[:,1])

accuracy_of_direction(y_test[:,0],x_test,y_test[:,1],attribCols,indexOfPrice=1)


In [39]:
def run_predictions_regression(df,prediction_cols = [['Close']],label_col='Label',
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],
                               train_percentages=[0.1,0.2,0.3,0.5,0.6,0.8],
                               window_sizes = [3,5,10,15,20,25,30,40,50,55,60,63,65,70,80,90,100,110,120,130,150],
                               csvPrefix='results_regression'                               
                              ):    
    ts_acc_best=-2.0
    best=[]
    resultsReg=[]
    for attribIndex in range(0,len(attribsSet)):            
        modelList = []
        modelList.append(("LinearRegression",LinearRegression()))
        ts_acc_best=-2.0
        best=[]
        results={}        

        for train_per in train_percentages:            
            for window in window_sizes:
                X,YCombo,Z = create_dataset_reg(df,attribsSet,attribIndex,
                            prediction_cols,pindex=0,label_col=label_col,window=window)
                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                X = min_max_scaler.fit_transform(X) 
                YCombo = min_max_scaler.fit_transform(YCombo) 

                x_train,x_test,y_train,y_test = train_test_split(X, YCombo, test_size=train_per, random_state=42)
                price_train = y_train[:,0]
                price_test  = y_test[:,0]
                
                for name, regression in modelList: 

                    regression.fit(x_train,price_train)            
                    y_pred_test = regression.predict(x_test)                                
                    ts_acc = accuracy_of_direction(y_pred_test,x_test,price_test,attribsSet[attribIndex],0)                
                    resultsReg.append([attribsSet[attribIndex],train_per,window,name,'N',ts_acc])
                    if(ts_acc_best <= ts_acc) :
                            ts_acc_best = ts_acc
                            best = [train_per,window,attribsSet[attribIndex],'No PCA']
                                                

                    pca_final = PCA(0.95)
                    df_train_pca = pca_final.fit_transform(x_train)
                    df_test_pca = pca_final.transform(x_test)
                    regression.fit(df_train_pca,price_train)            
                    y_pred_test_pca = regression.predict(df_test_pca)           
                    print(y_pred_test_pca.shape,price_test.shape)
                    ts_acc_pca = accuracy_of_direction(y_pred_test_pca,x_test,price_test,attribsSet[attribIndex],0)                
                    resultsReg.append([attribsSet[attribIndex],train_per,window,name,'Y',ts_acc_pca])

                    if(ts_acc_best <= ts_acc_pca) :
                            ts_acc_best = ts_acc_pca
                            best = [train_per,window,attribsSet[attribIndex],'PCA']

                    print('Completed ',name,attribsSet[attribIndex],train_per,window,'Acc ',
                          ts_acc,'Acc PCA components ',pca_final.n_components_,ts_acc_pca)
    save_results_to_csv(resultsReg,
                    ['Attributes','Train Per','Window size','Classifer','Using PCA','Accuracy']
                    ,csvPrefix)
                
    print('Best ',best,'Acc',ts_acc_best)                      

In [40]:
run_predictions_regression(df,
                          csvPrefix='results_regression_tplus1_')

Score 0.7386934673366834 147 199
(199,) (199,)
Score 0.6633165829145728 132 199
Completed  LinearRegression ['Close'] 0.1 3 Acc  0.7386934673366834 Acc PCA components  1 0.6633165829145728
Score 0.7788944723618091 155 199
(199,) (199,)
Score 0.47738693467336685 95 199
Completed  LinearRegression ['Close'] 0.1 5 Acc  0.7788944723618091 Acc PCA components  1 0.47738693467336685
Score 0.7171717171717171 142 198
(198,) (198,)
Score 0.47474747474747475 94 198
Completed  LinearRegression ['Close'] 0.1 10 Acc  0.7171717171717171 Acc PCA components  1 0.47474747474747475
Score 0.7777777777777778 154 198
(198,) (198,)
Score 0.5151515151515151 102 198
Completed  LinearRegression ['Close'] 0.1 15 Acc  0.7777777777777778 Acc PCA components  1 0.5151515151515151
Score 0.7411167512690355 146 197
(197,) (197,)
Score 0.47715736040609136 94 197
Completed  LinearRegression ['Close'] 0.1 20 Acc  0.7411167512690355 Acc PCA components  1 0.47715736040609136
Score 0.7614213197969543 150 197
(197,) (197,)
Sc

Score 0.5025295109612141 298 593
Completed  LinearRegression ['Close'] 0.3 15 Acc  0.7504215851602024 Acc PCA components  1 0.5025295109612141
Score 0.7495769881556683 443 591
(591,) (591,)
Score 0.4957698815566836 293 591
Completed  LinearRegression ['Close'] 0.3 20 Acc  0.7495769881556683 Acc PCA components  1 0.4957698815566836
Score 0.7406779661016949 437 590
(590,) (590,)
Score 0.49322033898305084 291 590
Completed  LinearRegression ['Close'] 0.3 25 Acc  0.7406779661016949 Acc PCA components  1 0.49322033898305084
Score 0.7295918367346939 429 588
(588,) (588,)
Score 0.4812925170068027 283 588
Completed  LinearRegression ['Close'] 0.3 30 Acc  0.7295918367346939 Acc PCA components  1 0.4812925170068027
Score 0.705982905982906 413 585
(585,) (585,)
Score 0.48034188034188036 281 585
Completed  LinearRegression ['Close'] 0.3 40 Acc  0.705982905982906 Acc PCA components  1 0.48034188034188036
Score 0.7319587628865979 426 582
(582,) (582,)
Score 0.5120274914089347 298 582
Completed  Line

Score 0.49279050042408823 581 1179
Completed  LinearRegression ['Close'] 0.6 25 Acc  0.7319762510602206 Acc PCA components  1 0.49279050042408823
Score 0.7185374149659864 845 1176
(1176,) (1176,)
Score 0.49489795918367346 582 1176
Completed  LinearRegression ['Close'] 0.6 30 Acc  0.7185374149659864 Acc PCA components  1 0.49489795918367346
Score 0.7188034188034188 841 1170
(1170,) (1170,)
Score 0.4717948717948718 552 1170
Completed  LinearRegression ['Close'] 0.6 40 Acc  0.7188034188034188 Acc PCA components  1 0.4717948717948718
Score 0.7216494845360825 840 1164
(1164,) (1164,)
Score 0.4802405498281787 559 1164
Completed  LinearRegression ['Close'] 0.6 50 Acc  0.7216494845360825 Acc PCA components  1 0.4802405498281787
Score 0.719207579672696 835 1161
(1161,) (1161,)
Score 0.5021533161068045 583 1161
Completed  LinearRegression ['Close'] 0.6 55 Acc  0.719207579672696 Acc PCA components  1 0.5021533161068045
Score 0.696027633851468 806 1158
(1158,) (1158,)
Score 0.48013816925734026 556

Score 0.9896373056994818 191 193
Completed  LinearRegression ['Close', 'Volume'] 0.1 60 Acc  0.9844559585492227 Acc PCA components  15 0.9896373056994818
Score 0.9948186528497409 192 193
(193,) (193,)
Score 0.9948186528497409 192 193
Completed  LinearRegression ['Close', 'Volume'] 0.1 63 Acc  0.9948186528497409 Acc PCA components  16 0.9948186528497409
Score 0.9948186528497409 192 193
(193,) (193,)
Score 0.9948186528497409 192 193
Completed  LinearRegression ['Close', 'Volume'] 0.1 65 Acc  0.9948186528497409 Acc PCA components  16 0.9948186528497409
Score 1.0 192 192
(192,) (192,)
Score 0.9947916666666666 191 192
Completed  LinearRegression ['Close', 'Volume'] 0.1 70 Acc  1.0 Acc PCA components  17 0.9947916666666666
Score 0.9947643979057592 190 191
(191,) (191,)
Score 1.0 191 191
Completed  LinearRegression ['Close', 'Volume'] 0.1 80 Acc  0.9947643979057592 Acc PCA components  20 1.0
Score 0.9947368421052631 189 190
(190,) (190,)
Score 0.9947368421052631 189 190
Completed  LinearRegre

(576,) (576,)
Score 0.9895833333333334 570 576
Completed  LinearRegression ['Close', 'Volume'] 0.3 70 Acc  0.9965277777777778 Acc PCA components  17 0.9895833333333334
Score 0.9930191972076788 569 573
(573,) (573,)
Score 0.9912739965095986 568 573
Completed  LinearRegression ['Close', 'Volume'] 0.3 80 Acc  0.9930191972076788 Acc PCA components  19 0.9912739965095986
Score 0.9964912280701754 568 570
(570,) (570,)
Score 0.987719298245614 563 570
Completed  LinearRegression ['Close', 'Volume'] 0.3 90 Acc  0.9964912280701754 Acc PCA components  21 0.987719298245614
Score 0.9947089947089947 564 567
(567,) (567,)
Score 0.9911816578483245 562 567
Completed  LinearRegression ['Close', 'Volume'] 0.3 100 Acc  0.9947089947089947 Acc PCA components  23 0.9911816578483245
Score 0.9946808510638298 561 564
(564,) (564,)
Score 0.9929078014184397 560 564
Completed  LinearRegression ['Close', 'Volume'] 0.3 110 Acc  0.9946808510638298 Acc PCA components  25 0.9929078014184397
Score 0.9982174688057041 560

Score 0.9885964912280701 1127 1140
Completed  LinearRegression ['Close', 'Volume'] 0.6 90 Acc  0.9929824561403509 Acc PCA components  21 0.9885964912280701
Score 0.9911816578483245 1124 1134
(1134,) (1134,)
Score 0.9911816578483245 1124 1134
Completed  LinearRegression ['Close', 'Volume'] 0.6 100 Acc  0.9911816578483245 Acc PCA components  23 0.9911816578483245
Score 0.9911347517730497 1118 1128
(1128,) (1128,)
Score 0.9884751773049646 1115 1128
Completed  LinearRegression ['Close', 'Volume'] 0.6 110 Acc  0.9911347517730497 Acc PCA components  24 0.9884751773049646
Score 0.9919786096256684 1113 1122
(1122,) (1122,)
Score 0.9901960784313726 1111 1122
Completed  LinearRegression ['Close', 'Volume'] 0.6 120 Acc  0.9919786096256684 Acc PCA components  25 0.9901960784313726
Score 0.9910394265232975 1106 1116
(1116,) (1116,)
Score 0.9874551971326165 1102 1116
Completed  LinearRegression ['Close', 'Volume'] 0.6 130 Acc  0.9910394265232975 Acc PCA components  27 0.9874551971326165
Score 0.9918

(191,) (191,)
Score 0.5497382198952879 105 191
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 80 Acc  0.6963350785340314 Acc PCA components  2 0.5497382198952879
Score 0.7368421052631579 140 190
(190,) (190,)
Score 0.5157894736842106 98 190
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 90 Acc  0.7368421052631579 Acc PCA components  2 0.5157894736842106
Score 0.7195767195767195 136 189
(189,) (189,)
Score 0.5185185185185185 98 189
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 100 Acc  0.7195767195767195 Acc PCA components  2 0.5185185185185185
Score 0.6808510638297872 128 188
(188,) (188,)
Score 0.5372340425531915 101 188
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 110 Acc  0.6808510638297872 Acc PCA components  3 0.5372340425531915
Score 0.6684491978609626 125 187
(187,) (187,)
Score 0.5080213903743316 95 187
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.1 120 Acc  0.6684491978609626

(578,) (578,)
Score 0.5519031141868512 319 578
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 63 Acc  0.7110726643598616 Acc PCA components  2 0.5519031141868512
Score 0.7179930795847751 415 578
(578,) (578,)
Score 0.5224913494809689 302 578
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 65 Acc  0.7179930795847751 Acc PCA components  2 0.5224913494809689
Score 0.7361111111111112 424 576
(576,) (576,)
Score 0.5173611111111112 298 576
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 70 Acc  0.7361111111111112 Acc PCA components  2 0.5173611111111112
Score 0.7102966841186736 407 573
(573,) (573,)
Score 0.5532286212914486 317 573
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 80 Acc  0.7102966841186736 Acc PCA components  2 0.5532286212914486
Score 0.7052631578947368 402 570
(570,) (570,)
Score 0.5526315789473685 315 570
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.3 90 Acc  0.7052631578947368

Score 0.7139175257731959 831 1164
(1164,) (1164,)
Score 0.5214776632302406 607 1164
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.6 50 Acc  0.7139175257731959 Acc PCA components  2 0.5214776632302406
Score 0.7071490094745909 821 1161
(1161,) (1161,)
Score 0.533161068044789 619 1161
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.6 55 Acc  0.7071490094745909 Acc PCA components  2 0.533161068044789
Score 0.7046632124352331 816 1158
(1158,) (1158,)
Score 0.5302245250431779 614 1158
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.6 60 Acc  0.7046632124352331 Acc PCA components  2 0.5302245250431779
Score 0.7041522491349481 814 1156
(1156,) (1156,)
Score 0.5449826989619377 630 1156
Completed  LinearRegression ['Close', 'Volume', 'High', 'Low'] 0.6 63 Acc  0.7041522491349481 Acc PCA components  2 0.5449826989619377
Score 0.7168831168831169 828 1155
(1155,) (1155,)
Score 0.5255411255411255 607 1155
Completed  LinearRegression ['Close', 'Vol

In [44]:
def create_models(window,attribsSet,attribIndex,output_size=1):  
            result=[]
            ann_name='model_1_2hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*1.5), activation='relu'))                            
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_2_2hidden_tanh_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(int(window*1.5), activation='tanh'))                            
            model.add(Dense(output_size, activation='tanh'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_3_2hidden_softmax_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_4_2hidden_softmax_sgd'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='sgd', loss='mean_squared_error')
            result.append([ann_name,model])
            
            ann_name='model_5_2hidden_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='softmax', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*1.5), activation='softmax'))                            
            model.add(Dense(output_size, activation='softmax'))                        
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
             
            
            ann_name='model_6_4hidden_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='relu', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='relu'))  
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            
            ann_name='model_7_4hidden_tanh_relu_adam'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='relu'))
            model.add(Dense(int(window*2), activation='relu'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='relu'))            
            model.compile(optimizer='adam', loss='mean_squared_error')
            result.append([ann_name,model])
            
            
            ann_name='model_8_4hidden_tanh_softmax_adagrad'
            #building the model
            model = Sequential()
            model.add(Dense(window, activation='tanh', input_shape=(window*len(attribsSet[attribIndex]),)))
            model.add(Dense(int(window*1.5), activation='softmax'))
            model.add(Dense(int(window*2), activation='softmax'))                            
            model.add(Dense(int(window*2), activation='tanh'))  
            model.add(Dense(int(window*1.5), activation='tanh'))
            model.add(Dense(output_size, activation='softmax'))            
            model.compile(optimizer='adagrad', loss='mean_squared_error')
            result.append([ann_name,model])
            
            return result

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [43]:
def run_predictions_ann(df,prediction_cols = [['Close']],label_col='Label',
                        attribsSet=[['Close','Volume','High','Low']],
                        epochsList=[200,300],
                        train_percentages=[0.3,0.5,0.7],
                        window_sizes = [5,10,15,20,30,50,60,65,80,90,100,120,150],
                        csvPrefix='results_ann'):
    
    ts_acc_best=-2.0
    best=[]    
    output_size=1
    for attribIndex in range(0,len(attribsSet)):    
        
        #train_percentages=[0.3,0.5,0.7]    
        #window_sizes = [5,10,15,20,30,50,60,65,80,90,100,120,150]        
        for train_per in train_percentages:  
            resultsANN=[]            
            for window in window_sizes:
                X,YCombo,Z = create_dataset_reg(df,attribsSet,attribIndex,
                            prediction_cols,pindex=0,label_col=label_col,window=window)
                min_max_scaler = MinMaxScaler(feature_range=(0,1))
                X = min_max_scaler.fit_transform(X) 
                YCombo = min_max_scaler.fit_transform(YCombo) 
                x_train,x_test,y_train,y_test = train_test_split(X, YCombo, test_size=train_per, random_state=42)
                price_train = y_train[:,0]
                price_test  = y_test[:,0]
                                            
                start = time.time()
                print('Creating models for window size ',window)
                modelList = create_models(window,attribsSet,attribIndex,y_train.shape[1])
                print('Created models for window size ',window,' in ',(time.time()-start))
                for name, model in modelList:
                    for epoch in epochsList:
                        print('Running model ',name,' with epoch ',epoch,attribsSet[attribIndex],window,train_per)
                        start = time.time()
                        #training the model
                        #history = model.fit(x_train, y_train, epochs=50, batch_size = 2)
                        history = model.fit(x_train, y_train, validation_split=0.15, epochs=epoch, batch_size=40, verbose=0,shuffle=False)
                        #print(model.summary())
                        y_test_pred=model.predict(x_test)
                        #print(y_test_pred[:,0],y_test[:,0],y_test_pred[:,1],y_test[:,1])
                        ts_acc = accuracy_of_direction(y_test_pred,x_test,price_test,attribsSet[attribIndex],0) 
                        
                        print(ts_acc)
#                         print('Completed model ',name,' with epoch ',epoch,
#                               [attribsSet[attribIndex],train_per,window,name,epoch,ts_acc],
#                               'Time ',(time.time()-start)
#                              )

#                         resultsANN.append([attribsSet[attribIndex],train_per,window,name,epoch,ts_acc])
#                         if(ts_acc_best <= ts_acc) :
#                                 ts_acc_best = ts_acc
#                                 best = [train_per,window,attribsSet[attribIndex],name]    
#             save_results_to_csv(resultsANN,
#                         ['Attributes','Train Per','Window size','ModelName','Epoch','Accuracy']
#                         ,csvPrefix+str(train_per))                      

#    print('Best ',best,'Acc',ts_acc_best)  
    

In [46]:
run_predictions_ann(df,epochsList=[200],
                        train_percentages=[0.5,0.7],
                        window_sizes = [60],
                          csvPrefix='results_ann_tplus1_')

Creating models for window size  60
Created models for window size  60  in  6.36202597618103
Running model  model_1_2hidden_relu_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.38238341968911915 369 965
0.38238341968911915
Running model  model_2_2hidden_tanh_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.6341968911917099 612 965
0.6341968911917099
Running model  model_3_2hidden_softmax_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.6176165803108808 596 965
0.6176165803108808
Running model  model_4_2hidden_softmax_sgd  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.6176165803108808 596 965
0.6176165803108808
Running model  model_5_2hidden_softmax_adagrad  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.6176165803108808 596 965
0.6176165803108808
Running model  model_6_4hidden_relu_adam  with epoch  200 ['Close', 'Volume', 'High', 'Low'] 60 0.5
Score 0.5533678756476684 534 965
0

In [ ]:
dfTplus5=df.dropna(subset=['Label5'])

print('Performing T plus 5')
run_predictions_classifier(df,prediction_cols = [['Label5']],attribsSet =[['Close','Volume','High','Low']],
                           train_percentages=[0.6],
                           window_sizes = [60],
                          csvPrefix='results_classification_tplus5_')

run_predictions_regression(dfTplus5,prediction_cols = [['Close5']],label_col='Label5',
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],                               
                               csvPrefix='results_regression_tplus5'                               
                              )

run_predictions_ann(df,prediction_cols = [['Close5']],label_col='Label5',epochsList=[200],
                        train_percentages=[0.5,0.7],
                        window_sizes = [60],
                          csvPrefix='results_ann_tplus5_')


In [ ]:
dfTplus20=df.dropna(subset=['Label20'])

print('Performing T plus 20')
run_predictions_classifier(df,prediction_cols = [['Label20']],attribsSet =[['Close','Volume','High','Low']],
                           train_percentages=[0.6],
                           window_sizes = [60],
                          csvPrefix='results_classification_tplus5_')

run_predictions_regression(dfTplus5,prediction_cols = [['Close20']],label_col='Label20',
                              attribsSet = [['Close'],['Close','Volume'],['Close','Volume','High','Low']],                               
                               csvPrefix='results_regression_tplus5'                               
                              )

run_predictions_ann(df,prediction_cols = [['Close20']],label_col='Label20',epochsList=[200],
                        train_percentages=[0.5,0.7],
                        window_sizes = [60],
                          csvPrefix='results_ann_tplus20_')